In [189]:
import pandas as pd
import numpy as np
from make_df import make_inputs #Custom module here
from naive import naive_fit #Custom module here
from survival_func import survival_fit #Custom module here


This is a demo of our Hard EM algorithm for survival analysis datasets
with a latent cured subpopulation. We will apply this to a melanoma data set and not only output the probability of being cured, but in addition the weights for the overall population survival function. 

STEP 1: ADD CENSORING INDICATOR COLUMN TO DATASET

In [13]:
mel = pd.read_csv('melanoma.csv')

In [14]:
mel.head()

,Unnamed: 0,time,status,sex,age,year,thickness,ulcer
0,1,10,3,1,76,1972,6.76,1
1,2,30,3,1,56,1968,0.65,0
2,3,35,2,1,41,1977,1.34,0
3,4,99,3,0,71,1968,2.90,0
4,5,185,1,1,52,1965,12.08,1


For the status column 1 means died from the disease, 2 means alive at then end of study, and 3 means died from unrelated
causes. Use this to make a censoring column as follows:
    
    1 means not censored (label 1)
    2 means censored (label 0)
    3 means cenosred (label 0)

In [15]:
censoring_indicator = mel['status'] 

In [16]:
censoring_indicator = censoring_indicator.replace(3, 0)

In [17]:
censoring_indicator = censoring_indicator.replace(2, 0)

In [18]:
mel['status'] = censoring_indicator

In [19]:
mel.head(10) #Sanity check

,Unnamed: 0,time,status,sex,age,year,thickness,ulcer
0,1,10,0,1,76,1972,6.76,1
1,2,30,0,1,56,1968,0.65,0
2,3,35,0,1,41,1977,1.34,0
3,4,99,0,0,71,1968,2.90,0
4,5,185,1,1,52,1965,12.08,1
5,6,204,1,1,28,1971,4.84,1
6,7,210,1,1,77,1972,5.16,1
7,8,232,0,0,60,1974,3.22,1
8,9,232,1,1,49,1968,12.88,1
9,10,279,1,0,68,1971,7.41,1


STEP 2: PREPARE DATA TO BE FED INTO CUSTOM ALGORITHM TO GET WEIGHTS FOR OVERALL SURVIVAL FUNCTION

In [206]:
def prepare_data(mel):
    
    censored_time = mel[mel.status==0].time
    noncensored_time = mel[mel.status==1].time
    covariates = ['sex', 'age', 'thickness', 'ulcer'] #Omit time, year, and status
    mel_covariates = mel[covariates] 
    mel_covariates=(mel_covariates-mel_covariates.mean())/mel_covariates.std() #standardize col. wise.
    mel_covariates['status'] = censoring_indicator
    
    #Extract censored inputs (status label 0) and noncensored inputs 
    #(status label 1) using our own function 

    columns = ['status']

    censored_inputs = make_inputs(mel_covariates, 0, columns) #extract censored

    noncensored_inputs = make_inputs(mel_covariates, 1, columns) #extract noncensored
    
    fit = naive_fit(censored_inputs, noncensored_inputs, 'use_HardEM')
    
    y_pred = fit['pred'] #predicted label of the corresponding row 
        
    y_scores = fit['prob'] #This is the probability of *not* being cured
    
    censored_mel = mel_covariates[mel_covariates['status']==0]

    noncensored_mel = mel_covariates[mel_covariates['status']==1]
    
    censored_mel['time'] = censored_time
    
    noncensored_mel['time'] = noncensored_time
    
    final = pd.concat([censored_mel, noncensored_mel]) 

    final['predicted_prob_cured'] = 1-y_scores
    
    final_cov = final[[col for col in final.columns if col not in ['time', 'predicted_prob_cured']]]
    
    cens_cov = final_cov[final_cov.status==0]

    noncens_cov = final_cov[final_cov.status==1]
    
    final_time_prob_cens = (final[final.status==0])[['time', 'predicted_prob_cured']]

    final_time_prob_noncens = (final[final.status==1])[['time', 'predicted_prob_cured']]
    
    cens_cov = cens_cov.drop(columns=['status'])
    
    print('covariates are: {} in that order'.format(list(cens_cov.columns)))
    
    times_n_probs = final_time_prob_cens.values.tolist()
    
    covs = cens_cov.values.tolist()
    
    z = list(zip(times_n_probs, covs))
    
    censored_inputs = [[arr[0][0], arr[0][1], arr[1]] for arr in z]
    
    noncens_cov = noncens_cov.drop(columns=['status'])

    ntimes_n_probs = final_time_prob_noncens.values.tolist()

    noncens_covs = noncens_cov.values.tolist()

    zn = list(zip(ntimes_n_probs, noncens_covs))

    noncensored_inputs = [[arr[0][0], arr[0][1], arr[1]] for arr in zn]
    
    return censored_inputs, noncensored_inputs, final
        

    

In [208]:
def get_weights(mel, dataset=False):

    censored_inputs, noncensored_inputs, table = prepare_data(mel)

    arr, SEs = survival_fit(censored_inputs, noncensored_inputs, 0, 100, standard_errors=True)
    
    output = 'scaling: {}, shape: {}, gamma: {}, SEs: {}'.format(arr[0], arr[1], arr[2:], SEs)
    
    if dataset:
        
        return [output, table]
    
    else:
        
        return output
    

In [211]:
out = get_weights(mel, dataset=True)

C:\Users\macguyver011\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\macguyver011\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\macguyver011\Anaconda3\lib\site-packages\autograd\numpy\numpy_vjps.py:78: RuntimeWarning: overflow encountered in true_divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
C:\Users\macguyver011\Anaconda3\lib\site-packages\autograd\core.py:212: RuntimeWarn

covariates are: ['sex', 'age', 'thickness', 'ulcer'] in that order


C:\Users\macguyver011\Anaconda3\lib\site-packages\autograd\tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)


In [212]:
out[0]

'scaling: 2268.4039588957785, shape: 1.752076705258878, gamma: [ 0.29592851 -0.09571339  0.37684971  0.44095896], SEs: [3.01524908e+02 2.11521647e-01 1.98401957e-01 2.07922084e-01\n 1.17403005e-01 2.24797584e-01]'

In [213]:
out[1]

,sex,age,thickness,ulcer,status,time,predicted_prob_cured
0,1.259824,1.411768,1.297595,1.127628,0,10,0.226160
1,1.259824,0.212131,-0.766989,-0.882491,0,30,0.629574
2,1.259824,-0.687597,-0.533837,-0.882491,0,35,0.670354
3,-0.789890,1.111859,-0.006709,-0.882491,0,99,0.702111
7,-0.789890,0.452058,0.101420,1.127628,0,232,0.523938
...,...,...,...,...,...,...,...
136,-0.789890,-0.627615,-0.222966,1.127628,1,2467,0.614617
142,1.259824,-1.107470,0.209549,1.127628,1,2565,0.430629
148,1.259824,1.531731,-0.331095,-0.882491,1,2782,0.516710
153,-0.789890,0.092167,0.209549,1.127628,1,3042,0.540847
